# YOLOv5 ball detection
### References

* YOLOv5 repository - https://github.com/ultralytics/yolov5
* https://www.kaggle.com/code/eneszvo/yolov5-helmet-detection-train-and-inference

In [1]:
# !pip install moviepy -q

In [2]:
# %cd ../
# !mkdir tmp
# %cd tmp

In [3]:
# Download YOLOv5
# !git clone https://github.com/ultralytics/yolov5  # clone repo
# %cd yolov5
# # Install dependencies
# %pip install -qr requirements.txt  # install dependencies

# %cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.2+cu113 (NVIDIA GeForce RTX 3090)


In [4]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import subprocess

In [5]:
# %cd ../
# !cp -r input/dfl-yolov5l6-ball-detection/yolov5l6_trained_600images.pt tmp/yolov5/
# !cp -r input/dfl-bundesliga-data-shootout/clips/08fd33_0.mp4 tmp/yolov5/
# %cd tmp/yolov5/

# YOLOv5 Inference with clip/08fd33_0.mp4

In [6]:
%cd ./work/yolov5

/workdir/work/yolov5


In [7]:
!ls

08fd33_0.mp4		  data		       train.py
BALL_DET_SAMPLE_SAVECONF  detect.py	       tutorial.ipynb
CONTRIBUTING.md		  export.py	       utils
DFL			  hubconf.py	       val.py
LICENSE			  models	       yolov5l6_trained_600images.pt
README.md		  output_ball_det.mp4  {CFG.EXP_NAME}
__pycache__		  requirements.txt
classify		  setup.cfg


In [9]:
movie = "08fd33_0.mp4"

In [11]:
!python detect.py --img 1280 \
                  --weights yolov5l6_trained_600images.pt \
                  --source {movie} \
                  --project DFL

detect: weights=['yolov5l6_trained_600images.pt'], source=08fd33_0.mp4, data=data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=DFL, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
fatal: unsafe repository ('/workdir/work/yolov5' is owned by someone else)
To add an exception for this directory, call:

	git config --global --add safe.directory /workdir/work/yolov5
YOLOv5 🚀 2022-9-4 Python-3.7.12 torch-1.10.2+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)

Fusing layers... 
Model summary: 476 layers, 76118664 parameters, 0 gradients
video 1/1 (1/750) /workdir/work/yolov5/08fd33_0.mp4: 768x1280 1 ball, 22.2ms
video 1/1 (2/750) /workdir/work/yolov5/08fd33_0.mp4: 768x1280 1 ball, 22.1ms
video 1/1 (3/750) /workdir/work/yolov5/08

In [ ]:
# !cp -r ./DFL/exp/ /kaggle/working/

In [ ]:
!ls

08fd33_0.mp4		  data		       train.py
BALL_DET_SAMPLE_SAVECONF  detect.py	       tutorial.ipynb
CONTRIBUTING.md		  export.py	       utils
DFL			  hubconf.py	       val.py
LICENSE			  models	       yolov5l6_trained_600images.pt
README.md		  output_ball_det.mp4  {CFG.EXP_NAME}
__pycache__		  requirements.txt
classify		  setup.cfg


In [ ]:
import subprocess

cap = cv2.VideoCapture("./08fd33_0.mp4")

frames = []
while True:
    ret, img = cap.read()
    if ret == False:
        break
    frames.append(img)
    width = img.shape[1]
    height = img.shape[0]
cap.release()

video = cv2.VideoWriter("../work/output/tmp_ball_det.mp4", cv2.VideoWriter_fourcc(*'MP4V'),
                                25, (width,height))
for img in frames:
    video.write(img)

video.release()   

subprocess.run(["ffmpeg", "-i", "../work/output/tmp_ball_det.mp4", "-crf", "18", "-preset", "veryfast",
                "-vcodec","libx264", "../work/output/output_ball_det.mp4",])

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --e

CompletedProcess(args=['ffmpeg', '-i', '../work/output/tmp_ball_det.mp4', '-crf', '18', '-preset', 'veryfast', '-vcodec', 'libx264', '../work/output/output_ball_det.mp4'], returncode=1)

# Show result

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from IPython.display import Video

filename = "output_ball_det.mp4"
ffmpeg_extract_subclip("../work/output/output_ball_det.mp4", 0, 30, targetname=filename)
    
Video(filename, width=800, embed=True)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command returned an error


OSError: ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libav